In [ ]:
!pip install transformers --quiet

In [7]:
!id="1kATR41mQR97Cb5xDfpuEXywWZfcITz-y"; conf=$(wget --quiet --save-cookies /tmp/cookies.txt --keep-session-cookies --no-check-certificate 'https://docs.google.com/uc?export=download&id='$id -O- | sed -rn 's/.*confirm=([0-9A-Za-z_]+).*/\1\n/p'); wget --load-cookies /tmp/cookies.txt "https://docs.google.com/uc?export=download&confirm=$conf&id=$id" -O grafn.csv && rm -rf /tmp/cookies.txt

huggingface/tokenizers: The current process just got forked, after parallelism has already been used. Disabling parallelism to avoid deadlocks...
To disable this warning, you can either:
	- Avoid using `tokenizers` before the fork if possible
	- Explicitly set the environment variable TOKENIZERS_PARALLELISM=(true | false)
--2022-08-25 13:36:58--  https://docs.google.com/uc?export=download&confirm=t&id=1U6g7mfAuDL19deR6uUqI1T1taqpKH8rd
Resolving docs.google.com (docs.google.com)... 142.250.203.206, 2a00:1450:401b:80d::200e
Connecting to docs.google.com (docs.google.com)|142.250.203.206|:443... connected.
HTTP request sent, awaiting response... 303 See Other
Location: https://doc-10-34-docs.googleusercontent.com/docs/securesc/ha0ro937gcuc7l7deffksulhg5h7mbp1/igh3u93c4cgt2cgt0eg42if0s01mv4md/1661427375000/05536367513713019519/*/1U6g7mfAuDL19deR6uUqI1T1taqpKH8rd?e=download&uuid=6f9c9276-6a84-496f-8bcd-7408eb581dd2 [following]
--2022-08-25 13:36:59--  https://doc-10-34-docs.googleuserconten

In [3]:
ndata = data[["text","label"]]
ndata.to_csv("raw/grafn_lite.csv")

In [2]:
import pandas as pd
import numpy as np
from tqdm import tqdm
from sklearn.model_selection import RepeatedStratifiedKFold

data = pd.read_csv("../../raw/grafn.csv",sep=",")
print("labels\n",data["label"].value_counts())
print("shape  \n",data.shape)

bootstrap_size = 0

if bootstrap_size != 0:
    bootstrap_factor = bootstrap_size / data.shape[0]
    bootstrap = np.random.uniform(size=data.shape[0]) < bootstrap_factor
    data = data.iloc[bootstrap]

X, y = data["text"].astype(str).values, data["label"].values

rskf = RepeatedStratifiedKFold(n_splits=2, n_repeats=5, random_state=1410)
foldids = []
for fold_idx, (train, test) in tqdm(enumerate(rskf.split(X, y)), total=rskf.get_n_splits()):
    foldids.append((fold_idx,train,test))
    
print("shapes X",X.shape,"y", y.shape)

/tmp/ipykernel_4055/349775969.py:6: DtypeWarning: Columns (3,5,6,9,10,11,12,13,14,16,22) have mixed types. Specify dtype option on import or set low_memory=False.
  data = pd.read_csv("../../raw/grafn.csv",sep=",")


labels
 0    50931
1    12999
Name: label, dtype: int64
shape  
 (63930, 24)


100%|██████████████████████████████████████████| 10/10 [00:00<00:00, 503.92it/s]

shapes X (63930,) y (63930,)


In [3]:
import transformers
transformers.logging.set_verbosity_error()
from torch.utils.data import DataLoader 

from transformers import AutoTokenizer
from transformers import AutoModel
from transformers import TFDistilBertModel, DistilBertTokenizerFast

from sklearn.base import BaseEstimator, TransformerMixin
import torch
import numpy as np

from scipy.sparse import csr_matrix
from tqdm import tqdm

from sklearn.linear_model import LogisticRegression

device = torch.device('cuda') if torch.cuda.is_available() else torch.device('cpu')
print("using device:", device)

if "disilbert_model" not in locals():
    disilbert_tokenizer =  AutoTokenizer.from_pretrained("distilbert-base-uncased")
    disilbert_model = AutoModel.from_pretrained("distilbert-base-uncased")
    handle = disilbert_model.to(device)


class BERTEmbeddings(BaseEstimator, TransformerMixin):
    def __init__(self):
        self.tokenizer =  disilbert_tokenizer
        self.model = disilbert_tokenizer
        self.max_length = 256
        self.model_name = disilbert_model

    def fit(self, X=None, y=None):
        pass
    
    def encode(self, txt):
        return self.tokenizer(txt, max_length=self.max_length, 
                              truncation=True, padding=True, return_tensors="pt")

    def transform(self, X=None):
        dataloader = DataLoader(X, batch_size=4, shuffle=False)
        allembeds = []
        for batch in tqdm(dataloader):
            batchenc = disilbert_tokenizer(batch, max_length=256, truncation=True, padding=True, return_tensors="pt")
            input_ids = batchenc['input_ids'].to(device)
            attention_mask = batchenc['attention_mask'].to(device)
            batchout = disilbert_model(input_ids, attention_mask=attention_mask, output_hidden_states=True)
            embeds = [vec[0].cpu().detach().numpy() for vec in batchout[1][-1]]
            allembeds.extend(embeds)
        return csr_matrix(allembeds)


class BertHead(BaseEstimator, TransformerMixin):
    def __init__(self):
        self.head = LogisticRegression(class_weight='auto', max_iter=10000)

    def fit(self, X=None, y=None):
        self.head.fit(X, y)


    def transform(self, X=None):
        pass
    
    def predict(self, X=None):    
        return self.head.predict(X)
    
    def predict_proba(self, X=None):    
        return self.head.predict_proba(X)



using device: cuda


In [6]:
bert = BERTEmbeddings()
X_dstil_numpy = bert.transform(X).toarray()

100%|████████████████████████████████████████████████████████████████████████████████████████████████████████████| 15983/15983 [03:01<00:00, 87.87it/s]


In [9]:
import pickle
# SAVE
with open('grafn_BERTEmbeddings.pickle', 'wb') as handle:
    pickle.dump(X_dstil_numpy, handle, protocol=pickle.HIGHEST_PROTOCOL)

In [4]:
import pickle
# LOAD
with open('../../pickles/bertcls_embeddings/grafn_BERTEmbeddings.pickle', 'rb') as handle:
    X_dstil_numpy=pickle.load(handle)

In [5]:
from sklearn.metrics import log_loss,accuracy_score, balanced_accuracy_score, f1_score, precision_score, recall_score, roc_auc_score
from imblearn.metrics import geometric_mean_score
from IPython.display import clear_output

from sklearn.linear_model import LogisticRegression
from sklearn.metrics import classification_report

def log_los_nonorm(x,y,average=False):
    return log_loss(x,y,normalize=True)

scores = {
    'Accuracy': {'func': accuracy_score},
    'Balanced Accuracy': {'func': balanced_accuracy_score},
    'F1': {'func': f1_score},
    'Precision': {'func': precision_score},
    'Recall': {'func': recall_score},
    'Log-likelihood': {'func': log_los_nonorm},
    #'G-mean': {'func': geometric_mean_score}
}

for score_name, score_dict in scores.items():
    scores[score_name]["list"] = []
    scores[score_name]["lab"] = []


for fold,j in enumerate(foldids):
    train = foldids[fold][1]
    test = foldids[fold][2]
    xin, yin = X_dstil_numpy[train], np.array(y[train])
    cls = BertHead()
    
    cls.fit(xin, yin)
    y_pred = cls.predict(X_dstil_numpy[test])
    y_pred_proba = cls.predict_proba(X_dstil_numpy[test])
    
    for score_name, score_dict in scores.items():
        if score_name == "Log-likelihood":
            scorval=score_dict['func'](y[test], y_pred_proba)
            score_dict['list'].append(scorval)
            print(score_name, scorval)
        elif score_name in ["F1","Precision","Recall"]:
            scorvaln = score_dict['func'](y[test], y_pred, average=None)
            score_dict['lab'].append(scorvaln)
            scorval = score_dict['func'](y[test], y_pred, average="weighted")
            score_dict['list'].append(scorval)
            print(score_name, scorval, scorvaln)  
        else:
            scorval=score_dict['func'](y[test], y_pred)
            score_dict['list'].append(scorval)
            print(score_name, scorval)
    print()

Accuracy 0.9201313937118724
Balanced Accuracy 0.8589558670271413
F1 0.9186091268295702 [0.95047623 0.79376363]
Precision 0.9181211055266055 [0.93916277 0.83568634]
Recall 0.9201313937118724 [0.96206558 0.75584615]
Log-likelihood 0.1998107273818329

Accuracy 0.9172845299546379
Balanced Accuracy 0.8547444944589522
F1 0.9157266497180765 [0.94870601 0.78649871]
Precision 0.915171387804657 [0.93753834 0.82752761]
Recall 0.9172845299546379 [0.96014294 0.74934605]
Log-likelihood 0.20453454010072863

Accuracy 0.9200375410605349
Balanced Accuracy 0.8601000166140556
F1 0.9186310526995018 [0.95037471 0.79426916]
Precision 0.91810977684571 [0.93986406 0.83288319]
Recall 0.9200375410605349 [0.96112311 0.75907692]
Log-likelihood 0.2002399955958455

Accuracy 0.9201313937118724
Balanced Accuracy 0.8580210136177515
F1 0.9185245495425799 [0.95050887 0.79319563]
Precision 0.9180654228630056 [0.93862705 0.83749572]
Recall 0.9201313937118724 [0.96269536 0.75334667]
Log-likelihood 0.20160552186328945

Accur

In [7]:
numlabels = scores["F1"]["lab"][0].shape[0]
scores["F1"]["lab"][0].shape[0] 
head = "| %-20s | %-12s |" +  numlabels * " %-10s |" 
headv = ["Score", "Average"]
headv.extend(["Kat_"+str(i+1) for i in range(numlabels)])
row=head % tuple(headv)
#print("+"*len(row))
print(row)
#print("+"*len(row))

for score_name, score_dict in sorted(scores.items()) :
    if score_name == "Log-likelihood":
        headv = [score_name, np.mean(score_dict['list'])*1, np.std(score_dict['list'])*1]
    else:
        headv = [score_name, np.mean(score_dict['list'])*100, np.std(score_dict['list'])*100]
    
    for i in range(numlabels):
        if score_name == "Log-likelihood":
            head = "| %-20s | %3.2f ± %3.3f |" + numlabels * " %-10s |" 
            headv.append("-")
        elif score_name in ["F1","Precision","Recall"]:
            head = "| %-20s | %4.1f ± %3.3f |" + numlabels* " %4.1f ± %3.1f |"
            vals = [v[i] for v in scores[score_name]["lab"]]
            headv.append(np.mean(vals)*100)
            headv.append(np.std(vals)*100)
        else:
            head = "| %-20s | %4.1f ± %3.3f |" + numlabels * " %-10s |" 
            headv.append("-")
    print(head % tuple(headv))
    
#print("+"*len(row))

| Score                | Average      | Kat_1      | Kat_2      |
| Accuracy             | 92.0 ± 0.116 | -          | -          |
| Balanced Accuracy    | 85.9 ± 0.186 | -          | -          |
| F1                   | 91.9 ± 0.117 | 95.0 ± 0.1 | 79.4 ± 0.3 |
| Log-likelihood       | 0.20 ± 0.002 | -          | -          |
| Precision            | 91.8 ± 0.120 | 93.9 ± 0.1 | 83.5 ± 0.4 |
| Recall               | 92.0 ± 0.116 | 96.2 ± 0.1 | 75.7 ± 0.3 |
